Best Practices
1. Preprocessing and cleaning
2. Train_Test_split
3. BPW,TFIDF,Word2vec
4. Choosing a ML algorithm

In [19]:
!pip install nltk
!pip install pandas
!pip install numpy
!pip install gensim
!pip install scikit-learn
!pip install bs4

In [51]:
import pandas as pd
df=pd.read_csv("all_kindle_review.csv")
df.head()


,Unnamed: 0.1,Unnamed: 0,asin,helpful,rating,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,0,11539,B0033UV8HI,"[8, 10]",3,"Jace Rankin may be short, but he's nothing to ...","09 2, 2010",A3HHXRELK8BHQG,Ridley,Entertaining But Average,1283385600
1,1,5957,B002HJV4DE,"[1, 1]",5,Great short read. I didn't want to put it dow...,"10 8, 2013",A2RGNZ0TRF578I,Holly Butler,Terrific menage scenes!,1381190400
2,2,9146,B002ZG96I4,"[0, 0]",3,I'll start by saying this is the first of four...,"04 11, 2014",A3S0H2HV6U1I7F,Merissa,Snapdragon Alley,1397174400
3,3,7038,B002QHWOEU,"[1, 3]",3,Aggie is Angela Lansbury who carries pocketboo...,"07 5, 2014",AC4OQW3GZ919J,Cleargrace,very light murder cozy,1404518400
4,4,1776,B001A06VJ8,"[0, 1]",4,I did not expect this type of book to be in li...,"12 31, 2012",A3C9V987IQHOQD,Rjostler,Book,1356912000


In [52]:
df=df[['reviewText','rating']]
df.head()

,reviewText,rating
0,"Jace Rankin may be short, but he's nothing to ...",3
1,Great short read. I didn't want to put it dow...,5
2,I'll start by saying this is the first of four...,3
3,Aggie is Angela Lansbury who carries pocketboo...,3
4,I did not expect this type of book to be in li...,4


In [53]:
df.shape

(12000, 2)

In [54]:
df['rating'].value_counts()

rating
5    3000
4    3000
3    2000
2    2000
1    2000
Name: count, dtype: int64

In [55]:
df['rating'].unique()

array([3, 5, 4, 2, 1], dtype=int64)

In [56]:
df['rating']=df['rating'].apply(lambda x : 0 if x<3 else 1)

In [57]:
df['rating'].isnull().sum()

0

In [58]:
df['reviewText'].isnull().sum()

0

In [59]:
df['reviewText']=df['reviewText'].str.lower()


In [60]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from bs4 import BeautifulSoup

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\GITESH\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [62]:
#Remove special characters
df['reviewText']=df['reviewText'].apply(lambda x: re.sub('[^a-z A-Z 0-9-]+','',x))
#Remove stopwords
df['reviewText']=df['reviewText'].apply(lambda x : " ".join([y for y in x.split() if y not in stopwords.words('english')]))
## Remove url 
df['reviewText']=df['reviewText'].apply(lambda x: re.sub(r'(http|https|ftp|ssh)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?', '' , str(x)))
# ## Remove html tags
# df['reviewText']=df['reviewText'].apply(lambda x: BeautifulSoup(x, 'lxml').get_text())
## Remove any additional spaces
df['reviewText']=df['reviewText'].apply(lambda x: " ".join(x.split()))


KeyboardInterrupt: 

In [63]:
df.head()

,reviewText,rating
0,jace rankin may short hes nothing mess man hau...,1
1,great short read didnt want put read one sitti...,1
2,ill start saying first four books wasnt expect...,1
3,aggie angela lansbury carries pocketbooks inst...,1
4,expect type book library pleased find price right,1


In [64]:
from nltk.stem import WordNetLemmatizer
lm=WordNetLemmatizer()

In [65]:
df['reviewText']=df['reviewText'].apply(lambda x: " ".join([lm.lemmatize(word) for word in x.split()]))

In [66]:
df.head()

,reviewText,rating
0,jace rankin may short he nothing mess man haul...,1
1,great short read didnt want put read one sitti...,1
2,ill start saying first four book wasnt expecti...,1
3,aggie angela lansbury carry pocketbook instead...,1
4,expect type book library pleased find price right,1


2. Train test split

In [67]:
from sklearn.model_selection import train_test_split
X_train_bow,X_test_bow,Y_train_bow,Y_test_bow=train_test_split(df['reviewText'],df['rating'],test_size=0.20,random_state=42)

In [68]:
X_train_bow.shape

(9600,)

In [78]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer()

In [79]:
X_train_bow_vector=cv.fit_transform(X_train_bow).toarray()
X_test_bow_vector=cv.transform(X_test_bow).toarray()

In [80]:
X_train_bow_vector.shape

(9600, 35726)

In [81]:
from sklearn.naive_bayes import GaussianNB
model_bow=GaussianNB()

In [82]:
model_bow.fit(X_train_bow_vector,Y_train_bow)

GaussianNB()

In [83]:
y_pred_bow=model_bow.predict(X_test_bow_vector)

In [84]:
from sklearn.metrics import accuracy_score,classification_report
print("Accuracy score: ",accuracy_score(Y_test_bow,y_pred_bow))
print("Classification report: ",classification_report(Y_test_bow,y_pred_bow))

Accuracy score:  0.5745833333333333
Classification report:                precision    recall  f1-score   support

           0       0.41      0.62      0.49       803
           1       0.74      0.55      0.63      1597

    accuracy                           0.57      2400
   macro avg       0.58      0.59      0.56      2400
weighted avg       0.63      0.57      0.59      2400

